在沙盒环境，使用期货多因子组合风险约束

In [1]:
import os,pdb,itertools,copy,datetime,sys,collections
os.environ['ULTRON_DATA'] = 'keim'

In [2]:
import numpy as np
import pandas as pd

In [3]:
from ultron.env import *
from ultron.strategy.builder.factor import ts_zscore_normalization,cross_section_scale
from ultron.strategy.builder.signal import calc_weighted_signal
from ultron.strategy.builder.risk import VolatilityRisk,TurnoverRisk,ExposureRisk,LeverageRisk

/var/log/ultron/2022-09-27.log


In [4]:
enable_example_env()

2022-09-27 19:28:52,389 - [env.py:67] - ultron - INFO - enable example env will only read /home/kerry/ultron/rom/sandbox/keim


#### 读取因子

In [5]:
total_data = pd.read_csv(os.path.join(g_project_data, 'factor.csv'), index_col=0)
total_data['trade_date'] = pd.to_datetime(total_data['trade_date'])
factors_data = total_data[['trade_date','code','WeightShortVolRelTotIntChg','BM_RecentSecond_20D']]
factors_data.head()

,trade_date,code,WeightShortVolRelTotIntChg,BM_RecentSecond_20D
0,2017-10-27,A,-0.037579,-0.023047
1,2017-10-27,AL,-0.012891,0.001133
2,2017-10-27,BU,0.275875,0.069999
3,2017-10-27,C,0.245555,-0.001541
4,2017-10-27,CF,-0.131799,0.004838


#### 时序标准化

In [6]:
def normalization(factors_data, name, bound=3, winsize=120):
    factor_data = factors_data.set_index(['trade_date','code'])[name].dropna().unstack()
    rval = ts_zscore_normalization(data=factor_data, bound=bound, winsize=winsize)
    return rval

In [7]:
factor_data1 = normalization(factors_data=factors_data, name='WeightShortVolRelTotIntChg')
factor_data2 = normalization(factors_data=factors_data, name='BM_RecentSecond_20D')

#### 横截面标准化

In [8]:
factor_data1 = cross_section_scale(factor_data1)
factor_data2 = cross_section_scale(factor_data2)

### 多因子信号合成

#### 构建数据集

In [9]:
signals = collections.OrderedDict()

In [10]:
signals['WeightShortVolRelTotIntChg'] = factor_data1
signals['BM_RecentSecond_20D'] = factor_data2
factor_list = ['WeightShortVolRelTotIntChg','BM_RecentSecond_20D']
factor_weight = [0.5,0.5]

#### 因子加权

In [11]:
weigthed = calc_weighted_signal(signals, factor_list, factor_weight)
weigthed.tail()

code,A,AL,BU,C,CF,CS,CU,EG,FU,HC,...,RU,SA,SF,SM,SR,TA,V,Y,ZC,ZN
trade_date,,,,,,,,,,,,,,,,,,,,,
2022-06-16,0.019153,0.041251,0.002103,-0.025657,0.032708,0.011506,0.012441,0.020483,-0.001438,-0.003308,...,0.018207,-0.020650,0.012860,-0.012399,0.038972,-0.038019,-0.026032,-0.007904,-0.008203,0.001723
2022-06-17,0.020092,0.031195,0.013501,-0.047116,0.019029,-0.008148,0.021857,0.004359,-0.034689,0.015959,...,0.041486,-0.022772,0.018861,0.002266,0.024402,-0.020301,-0.000903,-0.015956,-0.006656,0.019837
2022-06-20,0.028062,0.035740,0.014332,-0.026098,0.030309,-0.001631,0.035979,0.020550,-0.031629,0.023821,...,0.047478,-0.054444,0.025348,-0.007312,0.022329,-0.013262,-0.002426,-0.012271,0.003843,0.066617
2022-06-21,0.028113,0.037144,0.024982,-0.013389,0.035868,0.002471,0.034907,-0.004691,-0.010700,0.006578,...,0.040327,-0.015019,0.030972,-0.001642,0.082832,-0.030429,-0.014258,-0.015524,-0.004235,0.064214
2022-06-22,0.029034,0.025558,0.023268,0.043909,0.036202,-0.006711,0.033628,-0.013291,-0.032340,0.000139,...,0.047920,-0.018767,0.018534,0.013492,0.084965,-0.031115,-0.005580,-0.013369,0.001593,0.067409


### 加载行情

In [12]:
market_data = pd.read_csv(os.path.join(g_project_data, 'market_data.csv'), index_col=0)
market_data['trade_date'] = pd.to_datetime(market_data['trade_date'])
market_data.head()

,trade_date,code,openPrice,highestPrice,lowestPrice,closePrice,turnoverVol
0,2017-10-27,A,4462.578191,4463.801485,4413.646412,4435.665713,158774
1,2017-10-27,AL,15625.658581,15658.904663,15430.931529,15449.929290,293630
2,2017-10-27,BU,3310.339921,3336.950371,3283.729472,3302.356787,461826
3,2017-10-27,C,2009.751001,2014.561895,2001.331936,2002.534660,375480
4,2017-10-27,CF,20517.496003,20531.174333,20408.069357,20449.104349,84032


In [13]:
def prev_returs_impl(price_data, key, name):
    price_tb = price_data[key].unstack()
    price_tb.fillna(method='pad', inplace=True)
    return_tb = np.log(price_tb / price_tb.shift(1))
    return_tb = return_tb.replace([np.inf, -np.inf], np.nan)
    return_tb = return_tb.stack().reindex(price_data.index)
    return_tb.name = name
    return return_tb

In [14]:
prev_rets = prev_returs_impl(
    market_data.set_index(['trade_date','code']),'closePrice','pev1_ret').reset_index()
prev_rets['trade_date'] = pd.to_datetime(prev_rets['trade_date'])
prev_rets = prev_rets.set_index(['trade_date', 'code'])['pev1_ret'].unstack()
prev_rets.tail()

code,A,AL,BU,C,CF,CS,CU,EG,FU,HC,...,RU,SA,SF,SM,SR,TA,V,Y,ZC,ZN
trade_date,,,,,,,,,,,,,,,,,,,,,
2022-06-16,-0.001133,0.010048,-0.001924,0.002428,-0.015221,0.007933,-0.001841,-0.004387,-0.030516,-0.009501,...,0.001559,0.000688,-0.006102,-0.010734,-0.008543,-0.015461,-0.020520,-0.016212,0.000237,0.009448
2022-06-17,-0.004057,-0.010048,-0.009244,0.006215,-0.002303,0.005359,-0.011406,-0.008833,0.006671,-0.034433,...,-0.005467,-0.007935,-0.018534,-0.014328,-0.001684,-0.011396,-0.007270,-0.002287,0.024312,-0.010611
2022-06-20,0.004543,-0.015007,-0.048229,-0.001033,-0.014192,0.004391,-0.025266,-0.053275,-0.054396,-0.053743,...,0.003127,-0.005209,-0.040774,-0.026218,-0.003038,-0.006612,-0.025224,-0.023707,-0.038851,-0.019388
2022-06-21,-0.016154,0.013744,0.018415,-0.000689,-0.009137,0.000939,0.000441,-0.007350,0.017015,0.016685,...,-0.010200,-0.039778,-0.001926,0.004079,0.000507,-0.004626,-0.013116,-0.025762,0.013119,0.003159
2022-06-22,-0.011083,-0.021721,-0.030959,-0.014939,-0.009222,0.029881,-0.014361,-0.036307,-0.050863,-0.022626,...,-0.014297,-0.036529,-0.014078,-0.018486,-0.008654,-0.020493,-0.035797,-0.040992,0.000237,-0.002764


#### 波动率约束

In [15]:
volatiliy_risk = VolatilityRisk(volatility_winsize=250, volatility_base=100,
                                upper_limit=1, lower_limit=0.2)
rval = volatiliy_risk.run(portfolio=weigthed,
                          returns=prev_rets,
                          method='portfolio')
rval.tail()

code,A,AL,BU,C,CF,CS,CU,EG,FU,HC,...,RU,SA,SF,SM,SR,TA,V,Y,ZC,ZN
trade_date,,,,,,,,,,,,,,,,,,,,,
2022-06-16,0.019153,0.041251,0.002103,-0.025657,0.032708,0.011506,0.012441,0.020483,-0.001438,-0.003308,...,0.018207,-0.020650,0.012860,-0.012399,0.038972,-0.038019,-0.026032,-0.007904,-0.008203,0.001723
2022-06-17,0.020092,0.031195,0.013501,-0.047116,0.019029,-0.008148,0.021857,0.004359,-0.034689,0.015959,...,0.041486,-0.022772,0.018861,0.002266,0.024402,-0.020301,-0.000903,-0.015956,-0.006656,0.019837
2022-06-20,0.028062,0.035740,0.014332,-0.026098,0.030309,-0.001631,0.035979,0.020550,-0.031629,0.023821,...,0.047478,-0.054444,0.025348,-0.007312,0.022329,-0.013262,-0.002426,-0.012271,0.003843,0.066617
2022-06-21,0.028113,0.037144,0.024982,-0.013389,0.035868,0.002471,0.034907,-0.004691,-0.010700,0.006578,...,0.040327,-0.015019,0.030972,-0.001642,0.082832,-0.030429,-0.014258,-0.015524,-0.004235,0.064214
2022-06-22,0.029034,0.025558,0.023268,0.043909,0.036202,-0.006711,0.033628,-0.013291,-0.032340,0.000139,...,0.047920,-0.018767,0.018534,0.013492,0.084965,-0.031115,-0.005580,-0.013369,0.001593,0.067409


In [16]:
volatiliy_risk = VolatilityRisk(volatility_winsize=250, volatility_base=100,
                                upper_limit=1, lower_limit=0.2)
rval = volatiliy_risk.run(portfolio=weigthed,
                          returns=prev_rets,
                          method='product')
rval.tail()

code,A,AL,BU,C,CF,CS,CU,EG,FU,HC,...,RU,SA,SF,SM,SR,TA,V,Y,ZC,ZN
trade_date,,,,,,,,,,,,,,,,,,,,,
2022-06-16,0.019153,0.041251,0.002103,-0.025657,0.032708,0.011506,0.012441,0.020483,-0.001438,-0.003308,...,0.018207,-0.020650,0.012860,-0.012399,0.038972,-0.038019,-0.026032,-0.007904,-0.008203,0.001723
2022-06-17,0.020092,0.031195,0.013501,-0.047116,0.019029,-0.008148,0.021857,0.004359,-0.034689,0.015959,...,0.041486,-0.022772,0.018861,0.002266,0.024402,-0.020301,-0.000903,-0.015956,-0.006656,0.019837
2022-06-20,0.028062,0.035740,0.014332,-0.026098,0.030309,-0.001631,0.035979,0.020550,-0.031629,0.023821,...,0.047478,-0.054444,0.025348,-0.007312,0.022329,-0.013262,-0.002426,-0.012271,0.003843,0.066617
2022-06-21,0.028113,0.037144,0.024982,-0.013389,0.035868,0.002471,0.034907,-0.004691,-0.010700,0.006578,...,0.040327,-0.015019,0.030972,-0.001642,0.082832,-0.030429,-0.014258,-0.015524,-0.004235,0.064214
2022-06-22,0.029034,0.025558,0.023268,0.043909,0.036202,-0.006711,0.033628,-0.013291,-0.032340,0.000139,...,0.047920,-0.018767,0.018534,0.013492,0.084965,-0.031115,-0.005580,-0.013369,0.001593,0.067409


#### 换手率约束

In [17]:
turnover_risk = TurnoverRisk(grade_step=0.0008)
rval = turnover_risk.run(portfolio=weigthed)
rval.tail()

code,A,AL,BU,C,CF,CS,CU,EG,FU,HC,...,RU,SA,SF,SM,SR,TA,V,Y,ZC,ZN
trade_date,,,,,,,,,,,,,,,,,,,,,
2022-06-16,0.0184,0.0408,0.0016,-0.0256,0.0320,0.0112,0.0120,0.0200,-0.0008,-0.0032,...,0.0184,-0.0200,0.0128,-0.0120,0.0384,-0.0376,-0.0256,-0.0080,-0.0080,0.0016
2022-06-17,0.0200,0.0312,0.0128,-0.0464,0.0192,-0.0080,0.0216,0.0048,-0.0344,0.0152,...,0.0408,-0.0224,0.0184,0.0016,0.0248,-0.0208,-0.0016,-0.0152,-0.0072,0.0192
2022-06-20,0.0280,0.0352,0.0136,-0.0264,0.0296,-0.0024,0.0352,0.0200,-0.0320,0.0232,...,0.0472,-0.0544,0.0248,-0.0072,0.0224,-0.0136,-0.0024,-0.0128,0.0032,0.0664
2022-06-21,0.0280,0.0368,0.0248,-0.0136,0.0352,0.0024,0.0352,-0.0040,-0.0112,0.0072,...,0.0408,-0.0152,0.0304,-0.0024,0.0824,-0.0304,-0.0136,-0.0152,-0.0040,0.0648
2022-06-22,0.0288,0.0256,0.0240,0.0432,0.0360,-0.0064,0.0344,-0.0128,-0.0320,0.0008,...,0.0472,-0.0184,0.0192,0.0128,0.0848,-0.0304,-0.0056,-0.0136,0.0008,0.0672


#### 暴露约束

In [18]:
exposure_risk = ExposureRisk(commission_rate=0.0003,
                             portfolio_exposure=True,
                             max_portfolio=0.1,
                             product_exposure=False,
                             max_product=0.06,
                             turnover_winnratio_winsize=150)
rval = exposure_risk.run(portfolio=weigthed)
rval.tail()

code,A,AL,BU,C,CF,CS,CU,EG,FU,HC,...,RU,SA,SF,SM,SR,TA,V,Y,ZC,ZN
trade_date,,,,,,,,,,,,,,,,,,,,,
2022-06-16,0.019153,0.041251,0.002103,-0.017652,0.032708,0.011506,0.012441,0.020483,-0.000989,-0.002276,...,0.018207,-0.014207,0.012860,-0.008530,0.038972,-0.026157,-0.017910,-0.005438,-0.005644,0.001723
2022-06-17,0.020092,0.031195,0.013501,-0.044479,0.019029,-0.007692,0.021857,0.004359,-0.032748,0.015959,...,0.041486,-0.021498,0.018861,0.002266,0.024402,-0.019165,-0.000853,-0.015063,-0.006284,0.019837
2022-06-20,0.024031,0.030606,0.012273,-0.026098,0.025955,-0.001631,0.030811,0.017598,-0.031629,0.020400,...,0.040658,-0.054444,0.021707,-0.007312,0.019122,-0.013262,-0.002426,-0.012271,0.003291,0.057048
2022-06-21,0.020369,0.026912,0.018100,-0.013389,0.025988,0.001790,0.025291,-0.004691,-0.010700,0.004766,...,0.029218,-0.015019,0.022440,-0.001642,0.060014,-0.030429,-0.014258,-0.015524,-0.004235,0.046524
2022-06-22,0.017267,0.015200,0.013838,0.026114,0.021531,-0.006711,0.019999,-0.013291,-0.032340,0.000083,...,0.028499,-0.018767,0.011023,0.008024,0.050531,-0.031115,-0.005580,-0.013369,0.000947,0.040090


In [19]:
exposure_risk = ExposureRisk(commission_rate=0.0003,
                             portfolio_exposure=False,
                             max_portfolio=0.1,
                             product_exposure=True,
                             max_product=0.06,
                             turnover_winnratio_winsize=150)
rval = exposure_risk.run(portfolio=weigthed)
rval.tail()

code,A,AL,BU,C,CF,CS,CU,EG,FU,HC,...,RU,SA,SF,SM,SR,TA,V,Y,ZC,ZN
trade_date,,,,,,,,,,,,,,,,,,,,,
2022-06-16,0.019153,0.041251,0.002103,-0.017652,0.032708,0.011506,0.012441,0.020483,-0.000989,-0.002276,...,0.018207,-0.014207,0.012860,-0.008530,0.038972,-0.026157,-0.017910,-0.005438,-0.005644,0.001723
2022-06-17,0.020092,0.031195,0.013501,-0.044479,0.019029,-0.007692,0.021857,0.004359,-0.032748,0.015959,...,0.041486,-0.021498,0.018861,0.002266,0.024402,-0.019165,-0.000853,-0.015063,-0.006284,0.019837
2022-06-20,0.024031,0.030606,0.012273,-0.026098,0.025955,-0.001631,0.030811,0.017598,-0.031629,0.020400,...,0.040658,-0.054444,0.021707,-0.007312,0.019122,-0.013262,-0.002426,-0.012271,0.003291,0.057048
2022-06-21,0.020369,0.026912,0.018100,-0.013389,0.025988,0.001790,0.025291,-0.004691,-0.010700,0.004766,...,0.029218,-0.015019,0.022440,-0.001642,0.060000,-0.030429,-0.014258,-0.015524,-0.004235,0.046524
2022-06-22,0.017267,0.015200,0.013838,0.026114,0.021531,-0.006711,0.019999,-0.013291,-0.032340,0.000083,...,0.028499,-0.018767,0.011023,0.008024,0.050531,-0.031115,-0.005580,-0.013369,0.000947,0.040090


#### 杠杆约束

In [20]:
leverage_risk = LeverageRisk(risk_list=[],
                             weight_list=[],
                             risk_builder='riskweighted',
                             risk_control=True)
rval = leverage_risk.run(portfolio=weigthed)
rval.tail()

code,A,AL,BU,C,CF,CS,CU,EG,FU,HC,...,RU,SA,SF,SM,SR,TA,V,Y,ZC,ZN
trade_date,,,,,,,,,,,,,,,,,,,,,
2022-06-16,0.019153,0.041251,0.002103,-0.017652,0.032708,0.011506,0.012441,0.020483,-0.000989,-0.002276,...,0.018207,-0.014207,0.012860,-0.008530,0.038972,-0.026157,-0.017910,-0.005438,-0.005644,0.001723
2022-06-17,0.020092,0.031195,0.013501,-0.044479,0.019029,-0.007692,0.021857,0.004359,-0.032748,0.015959,...,0.041486,-0.021498,0.018861,0.002266,0.024402,-0.019165,-0.000853,-0.015063,-0.006284,0.019837
2022-06-20,0.024031,0.030606,0.012273,-0.026098,0.025955,-0.001631,0.030811,0.017598,-0.031629,0.020400,...,0.040658,-0.054444,0.021707,-0.007312,0.019122,-0.013262,-0.002426,-0.012271,0.003291,0.057048
2022-06-21,0.020369,0.026912,0.018100,-0.013389,0.025988,0.001790,0.025291,-0.004691,-0.010700,0.004766,...,0.029218,-0.015019,0.022440,-0.001642,0.060014,-0.030429,-0.014258,-0.015524,-0.004235,0.046524
2022-06-22,0.017267,0.015200,0.013838,0.026114,0.021531,-0.006711,0.019999,-0.013291,-0.032340,0.000083,...,0.028499,-0.018767,0.011023,0.008024,0.050531,-0.031115,-0.005580,-0.013369,0.000947,0.040090
